In [292]:
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import rotate
from data_preprocessing import RPEDataset
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset

In [293]:
original_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((149,149)),
    transforms.Compose([transforms.ToTensor(), torch.flatten])
])

cropped_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop((149, 149)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Compose([transforms.ToTensor(), torch.flatten])
])

In [294]:
rpe30 = RPEDataset(30, transform=original_transform)
rpe30_1 = RPEDataset(30, transform=cropped_transform)
rpe30_2 = RPEDataset(30, transform=cropped_transform)
rpe30_3 = RPEDataset(30, transform=cropped_transform)
rpe30_4 = RPEDataset(30, transform=cropped_transform)
rpe30_5 = RPEDataset(30, transform=cropped_transform)
rpe30_6 = RPEDataset(30, transform=cropped_transform)
rpe30_7 = RPEDataset(30, transform=cropped_transform)
rpe30_8 = RPEDataset(30, transform=cropped_transform)
rpe30_9 = RPEDataset(30, transform=cropped_transform)
rpe30_10 = RPEDataset(30, transform=cropped_transform)

rpe30set = ConcatDataset([rpe30, 
                          rpe30_1,
                          rpe30_2,
                          rpe30_3,
                          rpe30_4,
                          rpe30_5,
                          rpe30_6,
                          rpe30_7,
                          rpe30_8,
                          rpe30_9,
                          rpe30_10])

In [295]:
rpe45 = RPEDataset(45, transform=original_transform)
rpe45_1 = RPEDataset(45, transform=cropped_transform)
rpe45_2 = RPEDataset(45, transform=cropped_transform)
rpe45_3 = RPEDataset(45, transform=cropped_transform)
rpe45_4 = RPEDataset(45, transform=cropped_transform)

rpe45set = ConcatDataset([rpe45,
                          rpe45_1,
                          rpe45_2,
                          rpe45_3,
                          rpe45_4])

In [296]:
rpe60 = RPEDataset(60, transform=original_transform)
rpe60_1 = RPEDataset(60, transform=cropped_transform)
rpe60_2 = RPEDataset(60, transform=cropped_transform)
rpe60_3 = RPEDataset(60, transform=cropped_transform)

rpe60set = ConcatDataset([rpe60,
                         rpe60_1,
                         rpe60_2,
                         rpe60_3])

In [297]:
rpe180set = RPEDataset(180, transform=original_transform)

In [298]:
rpe330 = RPEDataset(330, transform=original_transform)
rpe330_1 = RPEDataset(330, transform=cropped_transform)
rpe330_2 = RPEDataset(330, transform=cropped_transform)

rpe330set = ConcatDataset([rpe330,
                           rpe330_1,
                           rpe330_2])

In [299]:
rpe720 = RPEDataset(720, transform=original_transform)
rpe720_1 = RPEDataset(720, transform=cropped_transform)
rpe720_2 = RPEDataset(720, transform=cropped_transform)
rpe720_3 = RPEDataset(720, transform=cropped_transform)
rpe720_4 = RPEDataset(720, transform=cropped_transform)
rpe720_5 = RPEDataset(720, transform=cropped_transform)
rpe720_6 = RPEDataset(720, transform=cropped_transform)

rpe720set = ConcatDataset([rpe720,
                           rpe720_1,
                           rpe720_2,
                           rpe720_3,
                           rpe720_4,
                           rpe720_5,
                           rpe720_6])

In [300]:
full_dataset = ConcatDataset([rpe30set,
                              rpe45set,
                              rpe60set,
                              rpe180set,
                              rpe330set,
                              rpe720set])

In [310]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [311]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [312]:
x_train = []
y_train = []
for data, labels in train_loader:
    x_train.append(data.numpy())
    y_train.append(labels.numpy())

x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

In [313]:
x_test = []
y_test = []
for data, labels in test_loader:
    x_test.append(data.numpy())
    y_test.append(labels.numpy())

x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [314]:
from sklearn import svm
model=svm.SVC(C=100, gamma=0.0001)

In [315]:
print('Training Started')
model.fit(x_train,y_train)
print('Training Finished')

Training Started
Training Finished


In [316]:
from sklearn.metrics import accuracy_score, classification_report

y_pred=model.predict(x_test)
print(f"The model is {accuracy_score(y_pred,y_test)*100:.4f}% accurate")
print(classification_report(y_test, y_pred, target_names=ages, zero_division=0))

The model is 73.0964% accurate
               precision    recall  f1-score   support

 30_day_crops       0.71      0.79      0.75        34
 45_day_crops       0.78      0.86      0.82        21
 60_day_crops       0.52      0.43      0.47        35
180_day_crops       0.85      0.94      0.89        35
330_day_crops       0.81      0.79      0.80        43
720_day_crops       0.65      0.59      0.62        29

     accuracy                           0.73       197
    macro avg       0.72      0.73      0.72       197
 weighted avg       0.72      0.73      0.72       197



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()

In [133]:
# saving a particular case of a model
import joblib

joblib.dump(model, 'RPE_svm_model.pkl')

['RPE_svm_model.pkl']

In [134]:
# loading a model via joblib

rpe_svm = joblib.load('RPE_svm_model.pkl')